# Accessing Qiskit Runtime on IBM Quantum

Before you can access IBM Quantum services, you need an IBM Quantum account. 
You can sign up for an account at https://quantum-computing.ibm.com/.

Once you have an account, you can grab your API token

### Authenticating with the server

In [1]:
# Import the module needed to access Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService

# Save account on disk.
# QiskitRuntimeService.save_account(channel="ibm_quantum", token=<IBM Quantum API token>)

# Load your IBM Quantum account or enable the account if it's not previously saved.
service   = QiskitRuntimeService(channel="ibm_quantum")
# service = QiskitRuntimeService(channel="ibm_quantum", token=<IBM Quantum API token>)

### Running the hello-world program

In [2]:
# Specify the backend name.
options = {"backend_name": "ibmq_qasm_simulator"}

job = service.run(
    program_id="hello-world",
    options=options,
    inputs={},
)

# Get the job result - this is blocking and control may not return immediately.
result = job.result()
print(result)

Hello, World!


# Calling Sampler inside a Runtime Session


1. Prepare initial input data
2. Select a backend
3. Open a session
4. Submit a request

## Step 1: preparing program inputs

The `Sampler` takes in the following arguments:

- **circuits**: a list of (parameterized) circuits that you want to investigate.
- **parameters**: a list of parameters for the parameterized circuits (optional).
- **skip_transpilation**: circuit transpilation is skipped if set to True. Default value is False.
- **service**: the `QiskitRuntimeService` instance to run the program on.
- **options**: Runtime options dictionary that control the execution environment.

You can also find the inputs and outputs of the primitives on the IBM Quantum website: https://quantum-computing.ibm.com/services?services=runtime&program=sampler 

### Prepare a parameterized circuit

**Transpilation** in Qiskit is the process of rewriting a given input circuit to

- convert high level gates to basis gates
- map the input circuit to match the topology of a specific quantum device
- optimize the circuit for execution

The transpiler makes things easier for the developers, but it can be time consuming. 

With VQA, circuit layout usually stays the same, and only the parameter values change. 

**Parameterized circuits** allow us to transpile once then bind with different parameters.

In [3]:
from qiskit.circuit.library import RealAmplitudes

pqc = RealAmplitudes(num_qubits=2, reps=2)
pqc.measure_all()

print(f"The circuit has {pqc.num_parameters} parameters")
pqc.decompose().draw()

The circuit has 6 parameters


┌──────────┐     ┌──────────┐     ┌──────────┐ ░ ┌─┐   
   q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├─░─┤M├───
        ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤ ░ └╥┘┌─┐
   q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├─░──╫─┤M├
        └──────────┘└───┘└──────────┘└───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════════════════════╩══╩═
                                                          0  1

## Step 2: Selecting a backend

You can find all the backends you have access to, real or simulated, on the IBM Quantum website: https://quantum-computing.ibm.com/services/resources

or programmatically:

In [5]:
# You can also check the backend configuration and properties.
# Backend configuration contains (mostly) static information about the backend.
# Backend properties contains dynamic information about the backend.

backend = service.least_busy(simulator=False, min_num_qubits=5)

print(f"Backend {backend.name} has {backend.configuration().num_qubits} qubits.")
print(f"T1 value for qubit 0 is {backend.properties().t1(0)}s")

Backend ibmq_belem has 5 qubits.
T1 value for qubit 0 is 0.00012124407978081647s


In [6]:
# In this workshop we will be using the simulator.

backend = "ibmq_qasm_simulator"

## Step 3: Opening a session

In [7]:
from qiskit_ibm_runtime import Sampler

# Use a context manager to open a session.
with Sampler(circuits=pqc, service=service, options={"backend": backend}) as sampler:
    pass

## Step 4: Submitting a request

In [8]:
with Sampler(circuits=pqc, service=service, options={"backend": backend}) as sampler:
    result = sampler(circuits=[0], parameter_values=[[0, 1, 1, 2, 3, 5]])
    print(result)

SamplerResult(quasi_dists=[{'10': 0.080078125, '00': 0.1103515625, '11': 0.439453125, '01': 0.3701171875}], metadata=[{'header_metadata': {}, 'shots': 1024}])


### Submitting multiple requests within a session

In [9]:
from qiskit.circuit.library import EfficientSU2

pqc2 = EfficientSU2(num_qubits=2, reps=1)
pqc2.measure_all()

print(f"The circuit has {pqc2.num_parameters} parameters")
pqc2.decompose().draw()

The circuit has 8 parameters


┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐ ░ ┌─┐   
   q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├─░─┤M├───
        ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤ ░ └╥┘┌─┐
   q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├─░──╫─┤M├
        └──────────┘└──────────┘└───┘└──────────┘└──────────┘ ░  ║ └╥┘
meas: 2/═════════════════════════════════════════════════════════╩══╩═
                                                                 0  1

In [10]:
from qiskit_ibm_runtime import Sampler

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [1, 2, 3, 4, 5, 6]
theta3 = [0, 1, 2, 3, 4, 5, 6, 7]

with Sampler(circuits=[pqc, pqc2], service=service, options={"backend": backend}) as sampler:    
    result = sampler(circuits=[0], parameter_values=[theta1], shots=2000)
    print(f"Result from circuit 0 with parameter {theta1}: {result}\n")
    
    result = sampler(circuits=[0, 0], parameter_values=[theta1, theta2])
    print(f"Result from circuit 0 with parameters {theta1, theta2}: {result}\n")
    
    result = sampler(circuits=[0, 1], parameter_values=[theta1, theta3])
    print(f"Result from circuits 0, 1 with parameters {theta1, theta2}: {result}")

Result from circuit 0 with parameter [0, 1, 1, 2, 3, 5]: SamplerResult(quasi_dists=[{'00': 0.1375, '10': 0.0905, '11': 0.415, '01': 0.357}], metadata=[{'header_metadata': {}, 'shots': 2000}])

Result from circuit 0 with parameters ([0, 1, 1, 2, 3, 5], [1, 2, 3, 4, 5, 6]): SamplerResult(quasi_dists=[{'10': 0.0955, '00': 0.1315, '11': 0.427, '01': 0.346}, {'01': 0.0325, '10': 0.585, '00': 0.0585, '11': 0.324}], metadata=[{'header_metadata': {}, 'shots': 2000}, {'header_metadata': {}, 'shots': 2000}])

Result from circuits 0, 1 with parameters ([0, 1, 1, 2, 3, 5], [1, 2, 3, 4, 5, 6]): SamplerResult(quasi_dists=[{'00': 0.1405, '10': 0.0965, '11': 0.42, '01': 0.343}, {'10': 0.143, '00': 0.031, '11': 0.671, '01': 0.155}], metadata=[{'header_metadata': {}, 'shots': 2000}, {'header_metadata': {}, 'shots': 2000}])


### Iterative processing

Prepare a parameterized circuit that takes 1 parameter, theta.

Theta determine the degree of rotation around the x axis (i.e. somewhere between straight up and straight down).

Starting from $ 0.25*pi $, we increment the value of theta by the probability of getting a 1 from the previous job times $ pi $.

For example, if the previous job has a distribution of `{'0': 0.65, '1': 0.35}`, the next job will run with theta=(0.35)*$pi$

The loop stops when the probability of getting a 1 reaches >= 50%

In [12]:
from qiskit.circuit import Parameter, QuantumCircuit

# Create a custom parameterized circuit.
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.measure_all()
qc.draw()

┌───────┐ ░ ┌─┐
     q: ┤ Rx(θ) ├─░─┤M├
        └───────┘ ░ └╥┘
meas: 1/═════════════╩═
                     0

In [14]:
from math import pi
from qiskit_ibm_runtime import Sampler

theta_val = 0.25*pi

with Sampler(circuits=[qc], service=service, options={"backend": backend}) as sampler:
    result = sampler(circuits=[0], parameter_values=[[theta_val]], shots=100)
    
    while result.quasi_dists[0].get('1', 0) < 0.5:
        print(f"Result is {result.quasi_dists} when using theta {theta_val}")
        
        theta_val += (result.quasi_dists[0].get('1', 0.25)*pi)
        result = sampler(circuits=[0], parameter_values=[[theta_val]], shots=100)

    print(f"All done. Result is {result.quasi_dists} when using theta {theta_val}")
        

Result is [{'1': 0.16, '0': 0.84}] when using theta 0.7853981633974483
Result is [{'1': 0.38, '0': 0.62}] when using theta 1.2880529879718152
All done. Result is [{'0': 0.14, '1': 0.86}] when using theta 2.4818581963359367


# ! Exercise !

1. Open a Qiskit Runtime session with the `Estimator` context manager.
2. Make multiple requests to the `estimator` primitive within the session.
    * The parameter values of a request is based on the previous one.
    * The loop ends when a target is achieved.

In [15]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright